# SemEval 2026 Task 8 - Task A: Retrieval

This notebook focuses on **Task A: Retrieval**. 
Goal: Given a question, retrieve relevant documents from the corpus.

**Output Format**:
The submission file must contain a `contexts` field (List of Objects), where each object has:
- `document_id`: The unique ID of the document.
- `score`: The relevance score.
- `text`: The document content (optional but recommended for debugging).

In [ ]:
import os
import json
import sys
from tqdm import tqdm
from typing import List, Dict, Any

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import RAG pipeline
try:
    from src.graph import app
except ImportError:
    print("ERROR: Run this notebook from the 'notebooks/' directory in the project structure.")

In [ ]:
# --- CONFIGURATION ---
TEAM_NAME = "Gbgers"
TASK_TYPE = "TaskA"
INPUT_FILE = "../dataset/human/generation_tasks/reference.jsonl"
OUTPUT_FILE = f"../data/submissions/submission_{TASK_TYPE}_{TEAM_NAME}.jsonl"

os.makedirs("../data/submissions", exist_ok=True)
print(f"Target: {OUTPUT_FILE}")

In [ ]:
def load_data(path):
    data = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                data.append(json.loads(line))
    return data

test_data = load_data(INPUT_FILE)
print(f"Loaded {len(test_data)} questions.")

In [ ]:
results = []

print("Running Retrieval Task...")
for item in tqdm(test_data):
    question = item.get("question")
    
    # Invoke Graph (will retrieve docs)
    try:
        # Start graph
        resp = app.invoke({"question": question})
        raw_docs = resp.get("documents", [])
    except Exception as e:
        print(f"Error on '{question}': {e}")
        raw_docs = []
    
    # Format Contexts
    contexts = []
    for doc in raw_docs:
        meta = getattr(doc, "metadata", {})
        contexts.append({
            "document_id": meta.get("id", meta.get("document_id", "unknown")),
            "text": getattr(doc, "page_content", ""),
            "score": float(meta.get("relevance_score", meta.get("score", 0.0)))
        })
    
    output_item = item.copy()
    output_item["contexts"] = contexts
    results.append(output_item)

print(f"Done. Generated {len(results)} entries.")

In [ ]:
# Save
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    for x in results:
        json.dump(x, f)
        f.write('\n')
print(f"Saved to {OUTPUT_FILE}")

# Simple Validation
sample = results[0]
if "contexts" in sample and isinstance(sample["contexts"], list):
    print("\033[92mVALIDATION PASS: Structure correct.\033[0m")
else:
    print("\033[91mVALIDATION FAIL: Missing 'contexts'.\033[0m")